# Surprise Singular Value Decomposition (SVD)

TODO: INTRO HERE...

## 0 Global Settings and Imports

In [1]:
import sys
sys.path.append("../../")
import surprise
import pandas as pd
from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import rmse, mae, rsquared, exp_var

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

System version: 3.6.6 (default, Sep  3 2018, 18:56:00) 
[GCC 8.2.1 20180831]
Surprise version: 1.0.6


In [2]:
# Parameters
# Add here if we need parameters


In [3]:
filepath = maybe_download("http://files.grouplens.org/datasets/movielens/ml-100k/u.data", "ml-100k.data")

In [4]:
data = pd.read_csv(filepath, sep="\t", names=["UserId", "MovieId", "Rating", "Timestamp"])
data = data[["UserId", "MovieId", "Rating"]]
data.head()

,UserId,MovieId,Rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [5]:
train, test = python_random_split(data)

In [6]:
train = surprise.Dataset.load_from_df(data, reader=surprise.Reader('ml-100k')).build_full_trainset()

In [7]:
svd = surprise.SVD().fit(train)

In [8]:
predictions = [svd.predict(row.UserId, row.MovieId, row.Rating) for (_, row) in test.iterrows()]

In [25]:
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(index=str, columns={'uid': 'User', 'iid': 'itemId', 'est': 'rating'})
predictions_df = predictions_df.drop(['details', 'r_ui'], axis='columns')

In [28]:
from reco_utils.evaluation.python_evaluation import mean_squared_error
mean_squared_error(test, predictions_df)

0.15199328893008127